<hr>
<font color='purple'>
    <font size="+3"><strong> 
Name: Theophilus Uwoghiren
</font>
</strong></font>

<br>
<hr>

<font color='green'>
    <font size="+3"><strong> 
Hamoye ID: 14843ce775c1f000
 </font>
 </strong></font>

<br>
<hr>

<font color='black'>
    <font size="+3">
Track: Data Science 
</font>         

<br>
<hr>
    
<font size="+3">
Stage C Assessment
</font>
<hr>

<br>
<font size="+3"><strong> 
Introduction
</strong>
</font>

**Stability of the Grid System**

Electrical grids require a balance between electricity supply and demand in order to be stable. Conventional systems achieve this balance through demand-driven electricity production. For future grids with a high share of inflexible (i.e., renewable) energy sources, the concept of demand response is a promising solution. This implies changes in electricity consumption in relation to electricity price changes. In this work, we’ll build a binary classification model to predict if a grid is stable or unstable using the UCI Electrical Grid Stability Simulated dataset.

Dataset: https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+m

It has 12 primary predictive features and two dependent variables.

**Predictive features:**

'tau1' to 'tau4': the reaction time of each network participant, a real value within the range 0.5 to 10 ('tau1' corresponds to the supplier node, 'tau2' to 'tau4' to the consumer nodes);
'p1' to 'p4': nominal power produced (positive) or consumed (negative) by each network participant, a real value within the range -2.0 to -0.5 for consumers ('p2' to 'p4'). As the total power consumed equals the total power generated, p1 (supplier node) = - (p2 + p3 + p4);
'g1' to 'g4': price elasticity coefficient for each network participant, a real value within the range 0.05 to 1.00 ('g1' corresponds to the supplier node, 'g2' to 'g4' to the consumer nodes; 'g' stands for 'gamma');

**Dependent variables:**

'stab': the maximum real part of the characteristic differential equation root (if positive, the system is linearly unstable; if negative, linearly stable);
'stabf': a categorical (binary) label ('stable' or 'unstable').

Because of the direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' should be dropped and 'stabf' will remain as the sole dependent variable (binary classification).

Split the data into an 80-20 train-test split with a random state of “1”. Use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). Use scikit learn to train a random forest and extra trees classifier. And use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.


**Data Set Information:**

The analysis is performed for different sets of input values using the methodology similar to that described in [SchÃ¤fer, Benjamin, et al. 'Taming instabilities in power grid networks by decentralized control.' The European Physical Journal Special Topics 225.3 (2016): 569-582.]. Several input values are kept the same: averaging time: 2 s; coupling strength: 8 s^-2; damping: 0.1 s^-1

**Attribute Information:**

11 predictive attributes, 1 non-predictive(p1), 2 goal fields:

    tau[x]: reaction time of participant (real from the range [0.5,10]s). Tau1 - the value for electricity producer.
    p[x]: nominal power consumed(negative)/produced(positive)(real). For consumers from the range [-0.5,-2]s^-2; p1 = abs(p2 + p3 + p4)
    g[x]: coefficient (gamma) proportional to price elasticity (real from the range [0.05,1]s^-1). g1 - the value for electricity producer.
    stab: the maximal real part of the characteristic equation root (if positive - the system is linearly unstable)(real)
    stabf: the stability label of the system (categorical: stable/unstable)

<br>
<font size="+3"><strong> 
Data Preprocessing
</strong>
</font>

**Importing Libraries**

In [1]:
# import pandas ans numpy library
import pandas as pd
import numpy as np

# import train_test_split to split the dataset into training and testing set
from sklearn.model_selection import train_test_split

# import StandardScaler
from sklearn.preprocessing import StandardScaler

# import accuracy_score, confusion_matrix, classification_report & f1_score to perform cross validation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

# import cross_val_score, RandomizedSearchCV, KFold & StratifiedKFold for measuring performance
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold, StratifiedKFold

# import RandomForestClassifier, ExtraTreesClassifier to create and select  Classifier model
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# import XGBClassifier for Gradient boost classifier
from xgboost import XGBClassifier

# import LigtGBM Classifier
from lightgbm import LGBMClassifier
import lightgbm as lgbm

import warnings

**Reading the Dataset**

In [2]:
#reading the dataset from the local file location

df = pd.read_csv('Data_for_UCI_named.csv')
print(df.shape)
df.head()

(10000, 14)


,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


**Comment:** The dataset contains 10000 rows and 14 columns

In [3]:
#Checking the Data Type for each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


**Comment:** Asides the 'stabf' column, looks like all other columns are numerical columns 

In [4]:
# checking for missing values

df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

**Comment:** None of the columns contain any missing values

In [5]:
#Some statistics for the numerical columns

df.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


**Data Cleaning**

In [6]:
#Dropping the 'stab' column as instructed

clean_df = df.copy().drop(columns = ["stab"])
clean_df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [7]:
clean_df.stabf.value_counts() #before split

unstable    6380
stable      3620
Name: stabf, dtype: int64

**Splitting the Data into feature matrix and target vector**

In [8]:
#Separating the Dataset into Independent and Dependent Features

X = clean_df.drop('stabf',axis=1)
y = clean_df['stabf']

print(X.shape, y.shape)

(10000, 12) (10000,)


**Splitting the Dataset into Training and Testing sets**

In [9]:
#Splitting the dataset into 80-20 train/test split with a random state of "1"

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
y_train.value_counts() # after split

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [11]:
y_test.value_counts() # after split

unstable    1288
stable       712
Name: stabf, dtype: int64

In [12]:
X_train.head() # before transformation

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
2694,6.255995,2.542401,7.024714,9.476518,3.529888,-1.224881,-0.688228,-1.616780,0.568221,0.618403,0.685739,0.660088
5140,5.070581,5.490253,8.075688,0.761075,4.220888,-1.280596,-1.902185,-1.038107,0.443515,0.097244,0.916955,0.129254
2568,1.220072,8.804028,3.874283,8.433949,3.614027,-1.039236,-0.953566,-1.621224,0.908353,0.923594,0.238881,0.660156
3671,7.498402,6.697603,8.798626,2.126236,3.134585,-1.581906,-0.589386,-0.963293,0.260826,0.899003,0.964752,0.600598
7427,7.074006,1.337511,6.100756,7.759156,2.526922,-0.922540,-0.632600,-0.971782,0.984580,0.716082,0.836928,0.165162


In [13]:
X_test.head() # before transformation

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
9953,6.877876,4.113820,9.356768,8.299753,4.056779,-1.897470,-1.590581,-0.568728,0.276567,0.845536,0.112440,0.822562
3850,5.802841,6.271371,4.731540,3.819867,3.579569,-1.709480,-1.067511,-0.802579,0.077527,0.416478,0.912846,0.861306
4962,2.286998,4.385142,2.830232,5.293880,3.035814,-1.202764,-0.902011,-0.931039,0.924216,0.130186,0.703887,0.063811
3886,5.019920,2.209962,6.266080,0.578901,4.322584,-1.960207,-1.074561,-1.287815,0.546910,0.065992,0.427349,0.814648
5437,7.646145,9.187896,5.484219,9.934313,3.634226,-1.254541,-1.335366,-1.044319,0.561528,0.121611,0.787318,0.300314


**Standardizing the Data**

In [14]:
#Using StanderScaler to transform the train_set and the test_set
std_scl = StandardScaler()
transform_X_train = pd.DataFrame(std_scl.fit_transform(X_train),columns = X_train.columns)
transform_X_test = pd.DataFrame(std_scl.transform(X_test),columns = X_test.columns)

In [15]:
transform_X_train.head() #after transformation

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


In [16]:
transform_X_train.head() #after transformation

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,0.367327,-0.986042,0.650447,1.547527,-0.291490,0.061535,1.293862,-0.845074,0.160918,0.339859,0.585568,0.492239
1,-0.064659,0.089437,1.035079,-1.641494,0.619865,-0.067235,-1.502925,0.486613,-0.293143,-1.558488,1.429649,-1.443521
2,-1.467850,1.298418,-0.502536,1.166046,-0.180521,0.490603,0.682560,-0.855302,1.399350,1.451534,-1.045743,0.492489
3,0.820081,0.529920,1.299657,-1.141975,-0.812854,-0.763632,1.521579,0.658780,-0.958319,1.361958,1.604140,0.275303
4,0.665424,-1.425627,0.312300,0.919137,-1.614296,0.760315,1.422019,0.639243,1.676895,0.695660,1.137504,-1.312575


<br>
<font size="+3"><strong> 
Answering the Quiz Questions
</strong>
</font>


**QUESTION 14**

What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

In [17]:
#random forest classifier

rfc = RandomForestClassifier(random_state=1)
rfc.fit(transform_X_train, y_train)
rfc_predict = rfc.predict(transform_X_test)
print('\nClassification Report:\n\n', classification_report(y_test, rfc_predict, digits=4))


Classification Report:

               precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000



In [18]:
#Accuracy Score

print('Accuracy Score:', round(accuracy_score(y_test, rfc_predict),4))

Accuracy Score: 0.929


**QUESTION 15**

What is the accuracy on the test set using the XGboost classifier? In 4 decimal places.

In [19]:
xgb = XGBClassifier(random_state=1)
xgb.fit(transform_X_train, y_train)
xgb_predict = xgb.predict(transform_X_test)

[14:51:30] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [20]:
print('\nClassification Report:\n\n', classification_report(y_test, xgb_predict, digits=4))


Classification Report:

               precision    recall  f1-score   support

      stable     0.9351    0.9101    0.9224       712
    unstable     0.9510    0.9651    0.9580      1288

    accuracy                         0.9455      2000
   macro avg     0.9430    0.9376    0.9402      2000
weighted avg     0.9453    0.9455    0.9453      2000



In [21]:
#Accuracy Score

print('Accuracy Score:', round(accuracy_score(y_test, xgb_predict),4))

Accuracy Score: 0.9455


**QUESTION 16**

What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.

In [22]:
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(transform_X_train, y_train)
lgbm_predict = lgbm.predict(transform_X_test)
print('\nClassification Report:\n\n', classification_report(y_test, lgbm_predict, digits=4))


Classification Report:

               precision    recall  f1-score   support

      stable     0.9276    0.9003    0.9138       712
    unstable     0.9458    0.9612    0.9534      1288

    accuracy                         0.9395      2000
   macro avg     0.9367    0.9307    0.9336      2000
weighted avg     0.9393    0.9395    0.9393      2000



In [23]:
#Accuracy Score

print('Accuracy Score:', round(accuracy_score(y_test, lgbm_predict),4))

Accuracy Score: 0.9395


**QUESTION 17**

Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

In [24]:
etc = ExtraTreesClassifier(random_state=1)
etc.fit(transform_X_train, y_train)
etc_predict = etc.predict(transform_X_test)
print('\nClassification Report:\n\n', classification_report(y_test, etc_predict, digits=4))


Classification Report:

               precision    recall  f1-score   support

      stable     0.9410    0.8511    0.8938       712
    unstable     0.9218    0.9705    0.9455      1288

    accuracy                         0.9280      2000
   macro avg     0.9314    0.9108    0.9197      2000
weighted avg     0.9287    0.9280    0.9271      2000



In [25]:
#Accuracy Score

print('Accuracy Score:', round(accuracy_score(y_test, etc_predict),4))

Accuracy Score: 0.928


In [26]:
n_estimators = [50, 100, 300, 500, 1000]    # Given we are to use this parameters to answer the question

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

In [27]:
# According to the parameters given to instantiate
rsv = RandomizedSearchCV(estimator = etc, param_distributions= hyperparameter_grid, random_state=1,cv = 5, n_iter=10,scoring='accuracy',n_jobs=1, verbose=1)

In [28]:
#checking for the best parameter for the model

search_result = rsv.fit(transform_X_train,y_train) 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [29]:
rsv.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

**QUESTION 18**

Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?


In [30]:
# experimenting with new hyperparameter
nh_etc_predict = rsv.predict(transform_X_test)

In [31]:
#classification report for this hyperparameter tuning

print('\nClassification Report:\n\n', classification_report(y_test, nh_etc_predict, digits=4))


Classification Report:

               precision    recall  f1-score   support

      stable     0.9211    0.8694    0.8945       712
    unstable     0.9300    0.9589    0.9442      1288

    accuracy                         0.9270      2000
   macro avg     0.9256    0.9141    0.9193      2000
weighted avg     0.9268    0.9270    0.9265      2000



In [32]:
#Accuracy Score

print('Accuracy Score:', round(accuracy_score(y_test, nh_etc_predict), 4))

Accuracy Score: 0.927


**Comment:**  Comparing both, the accuracy with the new hyperparameter is lower

**QUESTION 20**

Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least important respectively?


In [33]:
fi = pd.Series(etc.feature_importances_, X_train.columns).sort_values(ascending = True)
fi = pd.DataFrame(fi).reset_index()
fi.columns  = ["Features", "Importances"]
fi

,Features,Importances
0,p1,0.039507
1,p2,0.040371
2,p4,0.040579
3,p3,0.040706
4,g1,0.089783
5,g2,0.093676
6,g4,0.094019
7,g3,0.096883
8,tau3,0.113169
9,tau4,0.115466


**Comment:** 

Most important: tau2

Least Important: pi